Group Project Weather API Call

In [1]:
import os
from dotenv import load_dotenv
import mysql.connector
import requests
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import HTML, display
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from mysql.connector import Error

In [2]:
load_dotenv()

True

In [3]:
MYSQL_HOST = os.getenv('MYSQL_HOST')
MYSQL_USER = os.getenv('MYSQL_USER')
MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')
MYSQL_DATABASE = os.getenv('MYSQL_DATABASE')

conn = mysql.connector.connect(
    host=MYSQL_HOST,
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
    database=MYSQL_DATABASE
)

In [4]:
from datetime import datetime, timedelta

# Get today's date
today = datetime.today()

# Get the date 7 days from today
next_7_days = today + timedelta(days=6)

# Format the dates in the correct format (e.g., 'YYYY-MM-DD')
start_date = today.strftime('%Y-%m-%dT00:00:00Z')
end_date = next_7_days.strftime('%Y-%m-%dT23:59:59Z')

print("Start Date:", start_date)
print("End Date:", end_date)

Start Date: 2025-02-11T00:00:00Z
End Date: 2025-02-17T23:59:59Z


In [5]:
# Weather API
# Load the API key from environment variables
TOMORROW_API_KEY = os.getenv('TOMORROW_API_KEY')

# API pull with parameters
params = {
    "apikey": TOMORROW_API_KEY,       
    "city": "San Diego", 
    "startDateTime": start_date,  # Start date (today)
    "endDateTime": end_date      # End date (7 days later)                
}

# Define the API endpoint
TOMORROW_API_Endpoint = 'https://api.tomorrow.io/v4/weather/forecast?location=San%20Diego&fields=temperature_2m,weather_code&apikey=8zUbskgrkdcP29AtpCqw8XBCutnp5fJh'

# Send the request to the API
response = requests.get(TOMORROW_API_Endpoint, params=params)

In [6]:
# Check if the request was successful
if response.status_code == 200:
    print("API call succeeded!")
    data = response.json()  # The API response in JSON format
    print(data)             # Print the data (you'll see the events info here)
else:
    print(f"API call failed with status code: {response.status_code}")
    print("Response details:", response.text)  # Print more details on the failure

API call succeeded!
{'timelines': {'minutely': [{'time': '2025-02-12T06:21:00Z', 'values': {'cloudBase': 0.1, 'cloudCeiling': 0.1, 'cloudCover': 64, 'dewPoint': 19.3, 'freezingRainIntensity': 0, 'hailProbability': 22, 'hailSize': 4.61, 'humidity': 97, 'precipitationProbability': 0, 'pressureSeaLevel': 1015, 'pressureSurfaceLevel': 890, 'rainIntensity': 0, 'sleetIntensity': 0, 'snowIntensity': 0, 'temperature': 19.7, 'temperatureApparent': 19.7, 'uvHealthConcern': 0, 'uvIndex': 0, 'visibility': 10.66, 'weatherCode': 1102, 'windDirection': 305, 'windGust': 1.8, 'windSpeed': 0.5}}, {'time': '2025-02-12T06:22:00Z', 'values': {'cloudBase': 0.1, 'cloudCeiling': 0.1, 'cloudCover': 64, 'dewPoint': 19.3, 'freezingRainIntensity': 0, 'hailProbability': 21.8, 'hailSize': 6.11, 'humidity': 97, 'precipitationProbability': 0, 'pressureSeaLevel': 1015, 'pressureSurfaceLevel': 890, 'rainIntensity': 0, 'sleetIntensity': 0, 'snowIntensity': 0, 'temperature': 19.6, 'temperatureApparent': 19.6, 'uvHealthCo

In [7]:
# Extract temperature, humidity, rain, and weather code for each hour
weather_data = []

# Base URL for the weather icons from the GitHub repository
base_url = "https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/"

# Dictionary to map weather codes to icon URLs
weather_code_icons = {
    1100: f'{base_url}mostly_clear_day.svg',  # Example: mostly clear day
    1101: f'{base_url}partly_cloudy.svg',  # Example: partly cloudy
    1102: f'{base_url}mostly_cloudy.svg',  # Example: mostly cloudy
    1001: f'{base_url}cloudy.svg',  # Example: cloudy
    2100: f'{base_url}fog_light.svg',  # Example: light fog
    2000: f'{base_url}fog.svg',  # Example: fog
    4000: f'{base_url}drizzle.svg',  # Example: drizzle
    4200: f'{base_url}rain_light.svg',  # Example: light rain
    4001: f'{base_url}rain.svg',  # Example: rain
    4201: f'{base_url}rain_heavy.svg',  # Example: heavy rain
    6000: f'{base_url}freezing_drizzle.svg',  # Example: freezing drizzle
    6200: f'{base_url}freezing_rain_light.svg',  # Example: light freezing rain
    6001: f'{base_url}freezing_rain.svg',  # Example: freezing rain
    6201: f'{base_url}freezing_rain_heavy.svg',  # Example: heavy freezing rain
    8000: f'{base_url}tstorm.svg',  # Example: thunderstorm
  
}


for interval in data['timelines']['hourly']:
    time = interval['time']
    temperature = interval['values']['temperature']
    temperatureApparent = interval['values']['temperatureApparent']
    humidity = interval['values']['humidity']
    rain = interval['values']['rainIntensity']
    windSpeed = interval['values']['windSpeed']
    weather_codeday = interval['values']['weatherCode']
  
    # Get the icon URL for the weather code
    weather_icon = weather_code_icons.get(weather_codeday, f'{base_url}default.png')  # Use 'default.png' if code not found
    
    weather_data.append({
        'time': time,
        'temperature': temperature,
        'temperatureApparent': temperatureApparent,
        'humidity': humidity,
        'rain': rain,
        'windSpeed': windSpeed,
        'weather_code': weather_codeday,
        'weather_icon': weather_code_icons
    })

 

# Convert values to a DataFrame for easier visualization
weather_df = pd.DataFrame(weather_data)
print(weather_df)


# Function to display icons in the DataFrame
def path_to_image_html(path):
    return f'<img src="{path}" width="50" height="50">'

# Apply the function to the 'weather_icon' column
weather_df['weather_icon'] = weather_df['weather_icon'].apply(path_to_image_html)

# Display the DataFrame with icons
display(HTML(weather_df.to_html(escape=False)))

                     time  temperature  temperatureApparent  humidity  rain  \
0    2025-02-12T06:00:00Z         19.8                 19.8        97  0.00   
1    2025-02-12T07:00:00Z         18.1                 18.1        99  0.31   
2    2025-02-12T08:00:00Z         17.7                 17.7        97  0.00   
3    2025-02-12T09:00:00Z         17.8                 17.8        98  0.00   
4    2025-02-12T10:00:00Z         17.4                 17.4        95  0.00   
..                    ...          ...                  ...       ...   ...   
115  2025-02-17T01:00:00Z         23.9                 23.9        59  0.00   
116  2025-02-17T02:00:00Z         22.7                 22.7        65  0.00   
117  2025-02-17T03:00:00Z         21.6                 21.6        72  0.00   
118  2025-02-17T04:00:00Z         20.6                 20.6        81  0.00   
119  2025-02-17T05:00:00Z         19.6                 19.6        89  0.28   

     windSpeed  weather_code  \
0          0.6     

,time,temperature,temperatureApparent,humidity,rain,windSpeed,weather_code,weather_icon
0,2025-02-12T06:00:00Z,19.8,19.8,97,0.00,0.6,1102,"<img src=""{1100: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/mostly_clear_day.svg', 1101: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/partly_cloudy.svg', 1102: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/mostly_cloudy.svg', 1001: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/cloudy.svg', 2100: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/fog_light.svg', 2000: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/fog.svg', 4000: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/drizzle.svg', 4200: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/rain_light.svg', 4001: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/rain.svg', 4201: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/rain_heavy.svg', 6000: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/freezing_drizzle.svg', 6200: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/freezing_rain_light.svg', 6001: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/freezing_rain.svg', 6201: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/freezing_rain_heavy.svg', 8000: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/tstorm.svg'}"" width=""50"" height=""50"">"
1,2025-02-12T07:00:00Z,18.1,18.1,99,0.31,0.7,4000,"<img src=""{1100: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/mostly_clear_day.svg', 1101: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/partly_cloudy.svg', 1102: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/mostly_cloudy.svg', 1001: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/cloudy.svg', 2100: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/fog_light.svg', 2000: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/fog.svg', 4000: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/drizzle.svg', 4200: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/rain_light.svg', 4001: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/rain.svg', 4201: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/rain_heavy.svg', 6000: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/freezing_drizzle.svg', 6200: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/freezing_rain_light.svg', 6001: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/freezing_rain.svg', 6201: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/freezing_rain_heavy.svg', 8000: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/tstorm.svg'}"" width=""50"" height=""50"">"
2,2025-02-12T08:00:00Z,17.7,17.7,97,0.00,0.5,1101,"<img src=""{1100: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/mostly_clear_day.svg', 1101: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/partly_cloudy.svg', 1102: 'https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/mostly_cloudy.svg', 1001: 'https://raw.githubusercontent.com/tomorrow-io-api/

In [8]:
import mysql.connector
from mysql.connector import Error

# Function to connect to MySQL database
def connect_to_mysql():
    try:
        connection = mysql.connector.connect(
            host='MYSQL_HOST',
            database='MYSQL_DATABASE',
            user='MYSQL_USER',
            password='MYSQL_PASSWORD',
        )
        if connection.is_connected():
            print("Connected to MySQL database")
            return connection
    except Error as e:
        print(f"Error: {e}")
        return None

# Connect to the MySQL database
mysql_connection = connect_to_mysql()

# Check if the connection was successful
if mysql_connection:
    print("Connection to MySQL database was successful.")
else:
    print("Failed to connect to MySQL database.")


Error: 2005 (HY000): Unknown MySQL server host 'MYSQL_HOST' (11001)
Failed to connect to MySQL database.


In [14]:
# Function to insert data into MySQL
def insert_data_to_mysql(df, connection):
    try:
        if connection and connection.is_connected():
            cursor = connection.cursor()
            for row in df.itertuples(index=False):
                cursor.execute("""
                    INSERT INTO weather_data (time, temperature, temperature_apparent, humidity, rain, wind_speed, weather_code, weather_icon)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """, (row.time, row.temperature, row.temperatureApparent, row.humidity, row.rain, row.windSpeed, row.weather_code, row.weather_icon))
            connection.commit()
            print("Data inserted successfully")
    except Error as e:
        print(f"Error: {e}")
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")



In [15]:
# Insert the DataFrame into MySQL
if mysql_connection:
    insert_data_to_mysql(weather_df, mysql_connection)
else:
    print("Failed to connect to MySQL database. Data insertion aborted.")

Failed to connect to MySQL database. Data insertion aborted.
